In [ ]:
# Import all necessary packages

import json
import time

from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
api_dict = {}
version = ""

In [ ]:
try:
    # Use Firefox as the browser.
    # To use in linux, download geckodriver for firefox
    # https://sc.vern.cc/@seehleung/setup-geckodriver-and-chromedriver-for-selenium-in-linux-2e3f5f462c79
    driver = webdriver.Firefox()
    driver.get("https://developer.gst.gov.in/apiportal/taxpayer/returns/")

    # Wait till browser opens, page loads and the content populated
    time.sleep(10)

    # Click on 'View API List'
    element = driver.find_element(
        By.XPATH,
        "//a[@class='inverseLink id1' and @ng-click=\"apilist('returns','Returns')\"]",
    )
    element.click()

    # Wait for page to load
    time.sleep(5)

    # Loop over all 27 pages
    # TODO: Fetch page numbers from html
    #       Bit lazy to do it now
    for i in range(27):
        # Get all API items. For some reason, they are all inside an unordered tag
        ul_elements = driver.find_elements(By.CSS_SELECTOR, "ul.mar-t-10")
        for ul_element in ul_elements:
            # Click on the API link, this will open the api details
            anchor_tag = ul_element.find_element(By.TAG_NAME, "a")
            anchor_tag.click()
            time.sleep(3)

            api_dict[anchor_tag.text] = {}

            # Get API description
            desc = driver.find_element(By.CLASS_NAME, "text-justify")
            api_dict[anchor_tag.text]["description"] = desc.text

            # Populate api with its version details
            tbody = driver.find_element(By.TAG_NAME, "tbody")
            rows = tbody.find_elements(By.TAG_NAME, "tr")
            for row in rows:
                cells = row.find_elements(By.TAG_NAME, "td")
                for idx, cell in enumerate(cells):
                    if idx == 0:
                        version = cell.text
                    if idx == 0 and "apis" not in api_dict[anchor_tag.text]:
                        api_dict[anchor_tag.text]["apis"] = {}
                    if idx == 0 and "apis" in api_dict[anchor_tag.text] and version not in api_dict[anchor_tag.text]["apis"]:
                        api_dict[anchor_tag.text]["apis"][version] = []
                    else:
                        api_dict[anchor_tag.text]["apis"][version].append(cell.text)
        # Click on the next page
        element = driver.find_element(
            By.XPATH,
            "/html/body/div[1]/div/ng-view/div/div/div[3]/div/div/div/ul/li[11]/a",
        )
        element.click()

        # Wait till the page loads
        time.sleep(5)
finally:
    # Close the WebDriver
    driver.quit()

In [ ]:
processed_data = []
for api, api_details in api_dict.items():
    version_data = [
        dict(
            version=version,
            publishedOn=version_data[0],
            status=version_data[2],
        )
        for version, version_data in api_details["apis"].items()
    ]
    processed_data.append(
        dict(title=api, description=api_details["description"], apis=version_data)
    )

In [ ]:
with open("apis.json", "w") as f:
    json.dump(processed_data, f)